In [1]:
import requests

In [2]:
import bs4

In [3]:
import pandas as pd

In [4]:

#get https://rubinothings.com.br/player?id=6202

response = requests.get("https://rubinothings.com.br/player", params={"name": "rollabostx"})

from bs4 import BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html lang="pt-br">
 <head>
  <meta charset="utf-8"/>
  <title>
   RubinOThings - Player
  </title>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <meta content="Dashboard principal do RubinOThings com visão geral de personagens, nível médio, XP do dia e eventos recentes de todos os mundos RubinOT." name="description"/>
  <meta content="RubinOThings,RubinOT,online stats,exp gained today,event stream,xp dashboard, rubini ot, rubini hing, rubino things" name="keywords"/>
  <meta content="no-referrer-when-downgrade" name="referrer">
   <!-- Google AdSense (sitewide) -->
   <script async="" crossorigin="anonymous" src="https://pagead2.googlesyndication.com/pagead/js/adsbygoogle.js?client=ca-pub-7396771271432779">
   </script>
   <link href="favicon.png" rel="icon" type="image/png"/>
   <link href="xptracker.css?v=6" rel="stylesheet"/>
   <script src="https://cdn.jsdelivr.net/npm/chart.js">
   </script>
   <!-- Overrides visuais da página de jogado

In [5]:
def extract_tables(soup):
    """
    Extract all tables from BeautifulSoup object
    Returns a list of pandas DataFrames, each representing a table
    """
    dataframes = []
    
    # Find all table elements
    table_elements = soup.find_all('table')
    
    for i, table in enumerate(table_elements):
        headers = []
        rows = []
        
        # Extract headers
        header_elements = table.find_all('th')
        if header_elements:
            headers = [header.get_text(strip=True) for header in header_elements]
        else:
            # If no th elements, try to get headers from first tr
            first_row = table.find('tr')
            if first_row:
                first_row_cells = first_row.find_all(['td', 'th'])
                headers = [cell.get_text(strip=True) for cell in first_row_cells]
        
        # Extract all rows
        row_elements = table.find_all('tr')
        for row in row_elements:
            cells = row.find_all(['td', 'th'])
            if cells:
                row_data = [cell.get_text(strip=True) for cell in cells]
                # Skip if this row was used as headers
                if row_data != headers:
                    rows.append(row_data)
        
        # Create DataFrame
        if rows:
            if headers and len(headers) == len(rows[0]):
                df = pd.DataFrame(rows, columns=headers)
            else:
                # If no headers or header count doesn't match, use default column names
                df = pd.DataFrame(rows)
            
            # Add table index as an attribute
            df.attrs['table_index'] = i
            dataframes.append(df)
    
    return dataframes

In [6]:
# Extract all tables from the soup as DataFrames
tables = extract_tables(soup)

# Display information about found tables
print(f"Found {len(tables)} tables")
for i, df in enumerate(tables):
    print(f"\nTable {i + 1}:")
    print(f"  Shape: {df.shape}")
    print(f"  Columns: {list(df.columns)}")
    print(f"  Data:")
    print(df.head())  # Show first 5 rows
    print("  " + "="*50)

Found 4 tables

Table 1:
  Shape: (5, 4)
  Columns: ['Time', 'World', 'Level', 'Killed by']
  Data:
               Time    World Level       Killed by
0  23/12/2025 19:24  Auroria  1521     frostreaper
1  22/12/2025 01:55  Auroria  1519       sulphider
2  21/12/2025 15:18  Auroria  1519     frostreaper
3  10/12/2025 18:13  Auroria  1507       sulphider
4  03/12/2025 00:13  Auroria  1498  sopping corpus

Table 2:
  Shape: (10, 4)
  Columns: ['Time', 'World', 'Victim', 'Victim Level']
  Data:
               Time    World                     Victim Victim Level
0  23/12/2025 16:16  Auroria            Fator Suurpresa          658
1  23/12/2025 00:27  Auroria           Gauchao Bomba Um          783
2  19/12/2025 02:36  Auroria  Magnuszord Bomba Defender          894
3  15/12/2025 01:33  Auroria                Paco Hanshi         1217
4  14/12/2025 18:29  Auroria                     Bielrs         1488

Table 3:
  Shape: (20, 2)
  Columns: ['Date', 'Online time']
  Data:
         Date Online

In [7]:
#get https://rubinothings.com.br/status


#get last status updates
def get_last_status_updates():
    url = "https://rubinothings.com.br/status"
    response = requests.get(url)
    soup = bs4.BeautifulSoup(response.text, 'html.parser')
    
    r=extract_tables(soup)

    # Split the list of DataFrames by missing (all-NaN or all-empty) rows
    split_tables = []
    for df in r:
        # Find rows where all values are empty or NaN
        mask = df.apply(lambda row: all((str(x).strip() == '' or pd.isna(x)) for x in row), axis=1)
        # Get indices of such rows
        split_indices = mask[mask].index.tolist()
        # Split by those indices
        prev = 0
        for idx in split_indices:
            part = df.iloc[prev:idx]
            if not part.empty:
                split_tables.append(part.reset_index(drop=True))
            prev = idx + 1
        # Add the last part
        part = df.iloc[prev:]
        if not part.empty:
            split_tables.append(part.reset_index(drop=True))


    tables_dict = {}
    for table in split_tables:
        if not table.empty:
            table_name = str(table.iloc[0, 0]).strip()
            if "Status" in table_name:
                table_name = table_name.split("Status")[0].strip()
            new_table = table.iloc[1:].reset_index(drop=True)
            tables_dict[table_name] = new_table


    #extra cleaning
    
    #replace from first column the value "Rotine de coleta" to ""
    for key in tables_dict:
        df = tables_dict[key]
        if not df.empty:
            # Remove emojis and replace "Rotina de coleta" with ""
            df.iloc[:, 0] = (
                df.iloc[:, 0]
                .str.replace("Rotina de coleta", "")
                .str.replace(r"[^\w\s,.:-]", "", regex=True)
            )
            
            tables_dict[key] = df
            #set headers=["rotina",last update","time_outdated",status]
            if df.shape[1] >= 4:
                df.columns = ["rotina", "last update", "time_outdated", "status"]
                tables_dict[key] = df
    
    return tables_dict


r=get_last_status_updates()



In [8]:
r

{'Auroria':               rotina    last update time_outdated      status
 0  Daily Raw Ranking  Hoje às 09:23         5 min  Atualizado
 1             Online  Hoje às 09:29         0 min  Atualizado
 2        DeathsKills  Hoje às 09:24         5 min  Atualizado,
 'Belaria':               rotina    last update time_outdated      status
 0  Daily Raw Ranking  Hoje às 09:15        14 min  Atualizado
 1             Online  Hoje às 09:29         0 min  Atualizado
 2        DeathsKills  Hoje às 09:23         6 min  Atualizado,
 'Bellum':               rotina    last update time_outdated      status
 0  Daily Raw Ranking  Hoje às 09:26         2 min  Atualizado
 1             Online  Hoje às 09:29         0 min  Atualizado
 2        DeathsKills  Hoje às 09:23         5 min  Atualizado,
 'Elysian':               rotina    last update time_outdated      status
 0  Daily Raw Ranking  Hoje às 09:28         1 min  Atualizado
 1             Online  Hoje às 09:24         4 min  Atualizado
 2       

In [ ]:
def scrape_player_data(name):
    """
    Complete pipeline to scrape player data from rubinothings.com.br
    
    Args:
        name (str): Player name to search for
    
    Returns:
        dict: Dictionary containing:
            - 'name': The player name searched
            - 'tables': List of pandas DataFrames with all scraped tables
            - 'response_status': HTTP response status code
            - 'success': Boolean indicating if scraping was successful
    """
    result = {
        'name': name,
        'tables': [],
        'response_status': None,
        'success': False
    }
    
    try:
        # Make request to the API
        response = requests.get("https://rubinothings.com.br/player", params={"name": name})
        result['response_status'] = response.status_code
        
        if response.status_code == 200:
            # Parse HTML
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Extract all tables
            tables = extract_tables(soup)
            result['tables'] = tables
            result['success'] = True
            
            print(f"✅ Successfully scraped data for '{name}'")
            print(f"📊 Found {len(tables)} tables")
            
        else:
            print(f"❌ Request failed with status code: {response.status_code}")
            
    except Exception as e:
        print(f"❌ Error occurred: {str(e)}")
    
    return result

In [ ]:
# Use the pipeline to scrape data
player_name = "rollabostx"
data = scrape_player_data(player_name)

# Display results
if data['success']:
    print(f"\n🎯 Data for player: {data['name']}")
    print(f"📈 HTTP Status: {data['response_status']}")
    print(f"📊 Number of tables found: {len(data['tables'])}")
    
    # Display each table
    for i, df in enumerate(data['tables']):
        print(f"\n📋 Table {i + 1}:")
        print(f"   Shape: {df.shape}")
        print(f"   Columns: {list(df.columns)}")
        print("   Preview:")
        print(df.head(3).to_string(index=False))
        print("   " + "-"*40)
else:
    print(f"❌ Failed to scrape data for '{data['name']}'")
    print(f"Status code: {data['response_status']}")

In [ ]:
# Helper function to get specific table from results
def get_table(data, table_index=0):
    """
    Get a specific table from the scraped data
    
    Args:
        data (dict): Result from scrape_player_data function
        table_index (int): Index of the table to retrieve (0-based)
    
    Returns:
        pandas.DataFrame or None: The requested table or None if not found
    """
    if data['success'] and data['tables'] and table_index < len(data['tables']):
        return data['tables'][table_index]
    return None

# Example: Get the first table
first_table = get_table(data, 0)
if first_table is not None:
    print("📋 First table:")
    print(first_table)
else:
    print("❌ No tables found or invalid index")

In [9]:
# DATABASE INTEGRATION EXAMPLE
# This shows how to integrate the database system with your existing scraping functions

# First, install required packages (run once)
# pip install sqlalchemy pymysql

from alchemy import get_database_manager
from utils import process_character_data
import pandas as pd

# Initialize database connection
# Adjust these parameters for your MySQL server
db_manager = get_database_manager(
    host='localhost',
    port=3306,
    user='root',  # Your MySQL username
    password='your_password',  # Your MySQL password
    database='tibia_scraper'  # Database name (will be created if it doesn't exist)
)

# Create database tables (run this once)
print("🔧 Creating database tables...")
try:
    db_manager.create_tables()
    print("✅ Database tables created successfully!")
except Exception as e:
    print(f"❌ Error creating tables: {e}")
    print("Make sure MySQL server is running and credentials are correct")

ModuleNotFoundError: No module named 'pymysql'

In [ ]:
# INTEGRATE WITH YOUR EXISTING SCRAPING
# This cell shows how to use the database system with your existing functions

def scrape_and_store_character(character_name, world="Auroria"):
    """
    Complete function that scrapes character data and stores only new records in database
    """
    print(f"🔍 Scraping and storing data for: {character_name}")
    
    try:
        # Use your existing scraping function
        scraped_data = scrape_player_data(character_name)
        
        if not scraped_data['success']:
            print(f"❌ Failed to scrape data for {character_name}")
            return None
        
        # Get status updates using your existing function
        status_data = get_last_status_updates()
        
        # Process and store in database (only new records)
        results = process_character_data(
            db_manager=db_manager,
            character_name=character_name,
            world=world,
            scraped_data=scraped_data['tables'],  # Your scraped DataFrames
            status_data=status_data  # Status dict from scraping
        )
        
        print(f"✅ Stored {character_name}: {sum([results['new_deaths'], results['new_kills'], results['new_online_times'], results['new_experiences']])} new records")
        
        return results
        
    except Exception as e:
        print(f"❌ Error processing {character_name}: {e}")
        return None

# Example usage with your existing character
character_result = scrape_and_store_character("rollabostx")
print(f"Results: {character_result}")

In [7]:
t="""http://84.17.47.150:9002
http://84.17.47.149:9002
http://84.17.47.148:9002
http://84.17.47.147:9002
http://84.17.47.146:9002
http://84.17.47.126:9002
http://84.17.47.125:9002
http://84.17.47.124:9002
http://141.147.9.254:443
http://193.176.84.16:9002
http://193.176.84.19:9002
http://193.176.84.20:9002
http://193.176.84.21:9002
http://193.176.84.22:9002
http://193.176.84.23:9002
http://65.108.150.56:8443
http://194.67.91.153:443
http://129.151.160.199:443
http://197.243.20.178:443
http://31.28.4.192:443
http://167.99.124.118:443
http://4.188.236.47:443
http://78.28.152.111:443
http://196.223.129.21:443
http://34.122.187.196:443
http://103.37.111.253:10086
http://78.28.152.113:443
http://206.81.26.113:443
http://23.88.59.163:443
http://51.38.191.151:443
http://154.65.39.7:443
http://154.65.39.8:443
http://103.37.111.253:10089
http://207.178.166.187:443
http://103.179.190.121:443
http://116.203.117.22:443
http://65.108.104.111:443
http://184.168.123.21:443
http://128.199.207.200:443
http://144.91.90.109:443
http://45.114.142.178:443
http://34.68.168.129:443
http://195.114.209.50:443
http://138.199.35.215:9002
http://138.199.35.214:9002
http://138.199.35.213:9002
http://138.199.35.212:9002
http://138.199.35.208:9002
http://138.199.35.205:9002
http://138.199.35.204:9002
http://138.199.35.203:9002
http://138.199.35.201:9002
http://138.199.35.200:9002
http://138.199.35.198:9002
http://138.199.35.197:9002
http://138.199.35.196:9002
http://138.199.35.195:9002
http://45.92.108.112:443
http://193.176.84.24:9002
http://193.176.84.32:9002
http://193.176.84.34:9002
http://193.176.84.35:9002
http://193.176.84.37:9002
http://193.176.84.39:9002
http://193.176.84.40:9002
http://89.117.55.119:443
http://193.176.84.9:9002
http://193.176.84.10:9002
http://193.176.84.11:9002
http://193.176.84.12:9002
http://138.199.35.199:9002
http://138.199.35.217:9002
http://167.71.166.28:8443
http://57.129.1.214:443
http://156.146.59.28:9002
http://156.146.59.29:9002
http://156.146.59.50:9002
http://217.69.241.186:443
http://162.243.95.8:443
http://116.203.49.36:443
http://84.239.49.164:9002
http://156.146.59.8:9002
http://156.146.59.13:9002
http://156.146.59.2:9002
http://156.146.59.3:9002
http://156.146.59.4:9002
http://156.146.59.5:9002
http://156.146.59.6:9002
http://156.146.59.7:9002
http://156.146.59.9:9002
http://156.146.59.10:9002
http://84.239.49.37:9002
http://84.239.49.38:9002
http://84.239.49.39:9002
http://156.146.59.11:9002
http://84.239.49.40:9002
http://84.239.49.41:9002
http://84.239.49.43:9002
http://84.239.49.44:9002
http://84.239.49.45:9002
http://84.239.49.46:9002
http://84.239.49.47:9002
http://84.239.49.48:9002
http://84.239.49.49:9002
http://84.239.49.50:9002
http://156.146.59.12:9002
http://84.239.49.51:9002
http://156.146.59.14:9002
http://156.146.59.15:9002
http://156.146.59.16:9002
http://156.146.59.17:9002
http://156.146.59.18:9002
http://156.146.59.19:9002
http://156.146.59.20:9002
http://84.239.49.157:9002
http://84.239.49.161:9002
http://84.239.49.169:9002
http://84.239.49.200:9002
http://84.239.49.204:9002
http://84.239.49.209:9002
http://84.239.49.218:9002
http://84.239.49.220:9002
http://84.239.49.229:9002
http://84.239.49.231:9002
http://84.239.49.244:9002
http://84.239.49.246:9002
http://84.239.49.247:9002
http://84.239.49.248:9002
http://84.239.49.249:9002
http://84.239.49.250:9002
http://84.239.49.251:9002
http://84.239.49.253:9002
http://84.239.49.254:9002
http://84.239.14.176:9002
http://84.239.14.160:9002
http://84.239.14.146:9002
http://84.239.14.147:9002
http://84.239.14.148:9002
http://84.239.14.149:9002
http://84.239.14.150:9002
http://84.239.14.151:9002
http://84.239.14.152:9002
http://84.239.14.153:9002
http://84.239.14.154:9002
http://84.239.14.155:9002
http://84.239.14.156:9002
http://84.239.14.157:9002
http://84.239.14.158:9002
http://84.239.14.159:9002
http://84.239.14.162:9002
http://84.239.14.163:9002
http://84.239.14.164:9002
http://84.239.14.165:9002
http://84.239.14.166:9002
http://84.239.14.167:9002
http://84.239.14.168:9002
http://84.239.14.169:9002
http://84.239.14.170:9002
http://84.239.14.171:9002
http://84.239.14.172:9002
http://84.239.14.173:9002
http://84.239.14.174:9002
http://84.239.14.175:9002
http://192.73.244.36:443
http://51.254.213.34:443
http://156.232.10.14:3128
http://51.79.173.71:443
http://62.138.18.91:443
http://102.130.125.86:443
http://130.180.208.145:443
http://79.174.12.190:443
http://203.189.96.232:443
http://159.65.244.233:443
http://20.205.138.223:443
http://156.146.59.49:9002
http://84.239.49.154:9002
http://84.239.49.191:9002
http://84.239.49.238:9002
http://193.176.84.7:9002
http://156.146.59.41:9002
http://84.239.49.180:9002
http://84.239.49.245:9002
http://84.239.49.55:9002
http://84.239.49.199:9002
http://138.199.35.207:9002
http://84.239.49.242:9002
http://84.239.49.178:9002
http://193.176.84.36:9002
http://84.239.49.166:9002
http://138.199.35.220:9002
http://84.239.49.243:9002
http://84.239.49.211:9002
http://84.239.49.170:9002
http://84.239.49.62:9002
http://193.176.84.3:9002
http://84.239.49.60:9002
http://151.115.78.51:443
http://84.239.49.219:9002
http://138.199.35.216:9002
http://193.176.84.29:9002
http://193.176.84.8:9002
http://84.239.49.165:9002
http://84.239.49.230:9002
http://138.199.35.218:9002
http://193.176.84.33:9002
http://138.199.35.202:9002
http://84.239.49.162:9002
http://84.239.49.167:9002
http://84.239.49.163:9002
http://193.176.84.14:9002
http://84.239.49.224:9002
http://84.17.47.123:9002
http://84.239.49.188:9002
http://156.146.59.34:9002
http://84.239.49.160:9002
http://156.146.59.22:9002
http://156.146.59.27:9002
http://84.239.49.214:9002
http://37.9.171.157:443
http://84.239.49.232:9002
http://84.239.49.233:9002
http://156.146.59.23:9002
http://193.176.84.30:9002
http://84.239.49.175:9002
http://84.239.49.186:9002
http://156.146.59.39:9002
http://84.239.49.172:9002
http://84.239.49.177:9002
http://84.239.49.173:9002
http://84.239.49.201:9002
http://156.146.59.25:9002
http://156.146.59.31:9002
http://84.239.49.61:9002
http://84.239.49.221:9002
http://84.239.49.235:9002
http://84.239.49.240:9002
http://84.239.49.194:9002
http://84.239.49.185:9002
http://193.176.84.6:9002
http://84.239.49.159:9002
http://84.239.49.59:9002
http://84.239.49.236:9002
http://156.146.59.44:9002
http://193.176.84.4:9002
http://84.239.49.183:9002
http://84.239.49.168:9002
http://84.239.49.176:9002
http://193.176.84.38:9002
http://84.239.49.182:9002
http://138.199.35.219:9002
http://84.239.49.187:9002
http://84.239.49.56:9002
http://84.239.49.202:9002
http://156.146.59.32:9002
http://156.146.59.37:9002
http://84.239.49.207:9002
http://84.239.49.227:9002
http://84.239.49.179:9002
http://84.239.49.222:9002
http://138.199.35.206:9002
http://193.176.84.17:9002
http://193.176.84.13:9002
http://84.239.49.223:9002
http://84.239.49.210:9002
http://84.239.49.171:9002
http://156.146.59.33:9002
http://138.199.35.211:9002
http://84.239.49.203:9002
http://84.239.49.252:9002
http://156.146.59.48:9002
http://84.239.49.239:9002
http://84.239.49.54:9002
http://84.239.49.156:9002
http://84.239.49.205:9002
http://156.146.59.21:9002
http://156.146.59.35:9002
http://84.239.49.198:9002
http://84.239.49.225:9002
http://84.239.49.155:9002
http://156.146.59.40:9002
http://84.239.49.181:9002
http://84.239.49.226:9002
http://84.239.49.190:9002
http://156.146.59.36:9002
http://84.239.49.206:9002
http://84.239.49.189:9002
http://84.239.49.53:9002
http://84.239.49.228:9002
http://138.199.35.209:9002
http://193.176.84.26:9002
http://84.239.49.213:9002
http://84.239.49.234:9002
http://84.239.49.217:9002
http://47.88.35.91:3128
http://13.80.134.180:443
http://51.83.190.248:443
http://79.139.61.227:443
http://149.126.16.38:443
http://13.66.206.110:443
http://40.113.199.101:443
http://52.184.228.103:443
http://40.122.106.245:443
http://13.77.70.244:443
http://159.65.245.255:443
http://154.26.139.232:443
http://154.90.48.76:443
http://104.129.194.38:10228
http://51.158.253.115:443
http://51.158.253.130:443
http://51.158.253.120:443
http://51.158.253.113:443
http://51.158.253.153:443
http://51.158.253.152:443
http://156.146.59.46:9002
http://84.239.49.42:9002
http://156.146.59.42:9002
http://84.239.49.57:9002
http://84.239.49.58:9002
http://84.239.49.158:9002
http://84.239.49.184:9002
http://84.239.49.192:9002
http://84.239.49.193:9002
http://84.239.49.196:9002
http://84.239.49.197:9002
http://84.239.49.208:9002
http://84.239.49.212:9002
http://84.239.49.215:9002
http://84.239.49.241:9002
http://156.146.59.24:9002
http://156.146.59.30:9002
http://156.146.59.38:9002
http://156.146.59.43:9002
http://156.146.59.45:9002
http://156.146.59.47:9002
http://51.158.252.252:443
http://51.158.253.114:443
http://51.158.252.160:443
http://51.158.253.128:443
http://51.158.253.154:443
http://167.99.236.14:443
http://51.158.253.45:443
http://51.158.253.77:443
http://51.158.253.78:443
http://138.199.35.210:9002
http://128.199.195.19:8888
http://104.129.194.46:10228
http://104.129.194.44:10228
http://104.129.194.43:11263
http://104.129.194.38:11263
http://148.233.136.213:443
http://165.225.72.38:11745
http://165.225.72.38:11742
http://54.37.72.89:443
http://104.129.194.45:11723
http://47.243.181.85:8081
http://104.129.194.44:11265
http://104.129.194.38:11742
http://104.129.194.46:11742
http://104.129.194.44:11742
http://104.129.194.45:11724
http://104.129.194.43:11724
http://152.230.215.123:443
http://165.225.72.38:11723
http://104.129.194.44:11723
http://104.129.194.38:11723
http://104.129.194.44:11747
http://104.129.194.44:11677
http://103.241.129.229:8443
http://103.241.129.2:8443
http://103.241.129.6:8443
http://103.241.129.10:8443
http://103.241.129.12:8443
http://103.241.129.13:8443
http://103.241.129.16:8443
http://103.241.129.18:8443
http://103.241.129.23:8443
http://103.241.129.26:8443
http://103.241.129.27:8443
http://103.241.129.28:8443
http://103.241.129.181:8443
http://103.241.129.182:8443
http://103.241.129.196:8443
http://103.241.129.209:8443
http://103.241.129.217:8443
http://103.241.129.239:8443
http://103.241.129.240:8443
http://103.241.129.245:8443
http://103.241.129.246:8443
http://103.241.129.247:8443
http://103.241.129.249:8443
http://103.241.129.252:8443
http://103.241.130.0:8443
http://103.241.130.4:8443
http://103.241.129.11:8443
http://103.241.129.177:8443
http://103.241.129.192:8443
http://103.241.129.194:8443
http://103.241.129.197:8443
http://103.241.129.199:8443
http://103.241.129.200:8443
http://103.241.129.201:8443
http://103.241.129.226:8443
http://103.241.129.230:8443
http://103.241.129.231:8443
http://103.241.129.235:8443
http://103.241.129.243:8443
http://103.241.129.250:8443
http://103.241.129.208:8443
http://103.241.129.212:8443
http://103.241.129.216:8443
http://104.129.194.44:11749
http://104.129.194.46:11749
http://104.129.194.45:11480
http://104.129.194.43:11480
http://104.129.194.46:11480
http://104.129.194.38:11500
http://104.129.194.38:11728
http://104.129.194.38:10227
http://103.37.111.253:10087
http://104.129.194.38:11726
http://104.129.194.45:11065
http://104.129.194.46:11273
http://104.129.194.45:11500
http://104.129.194.38:11714
http://104.129.194.43:11703
http://165.225.72.38:11698
http://104.129.194.43:11712
http://104.129.194.46:11716
http://104.129.194.44:11703
http://104.129.194.38:11703
http://104.129.194.46:11741
http://104.129.194.43:11691
http://104.129.194.46:11714
http://104.129.194.46:11734
http://104.129.194.38:11677
http://104.129.194.45:11677
http://212.58.132.188:8443
http://104.129.194.45:11703
http://104.129.194.45:11714
http://104.129.194.45:11741
http://104.129.194.43:11716
http://104.129.194.46:11065
http://165.225.72.38:11771
http://104.129.194.43:11730
http://104.129.194.44:11711
http://165.225.72.38:11721
http://104.129.194.45:11691
http://165.225.120.33:10230
http://165.225.124.33:10230
http://64.181.235.123:3128
http://65.109.40.163:8443
http://23.106.56.43:19881
http://167.235.198.62:443
http://104.129.194.38:11682
http://51.161.79.152:8888
http://165.225.72.38:11734
http://165.225.72.38:11733
http://196.1.93.16:443
http://162.19.49.131:443
http://94.23.9.170:443
http://104.129.194.43:11725
http://51.159.225.129:8118
http://187.217.194.178:86
http://87.255.196.143:443
http://186.65.109.21:443
http://104.129.194.44:11748
http://136.226.232.13:10230
http://212.83.168.126:8081
http://51.159.226.157:443
http://51.159.226.159:443
http://51.159.226.153:443
http://51.159.226.154:443
http://51.159.226.151:443
http://51.159.226.150:443
http://51.159.226.155:443
http://51.159.226.149:443
http://51.159.226.156:443
http://51.159.226.162:443
http://51.159.226.128:443
http://51.159.226.160:443
http://51.159.226.158:443
http://51.159.226.161:443
http://51.159.226.152:443
http://104.129.194.44:11657
http://104.129.194.43:11657
http://165.225.72.38:11657
http://104.129.194.38:11657
http://51.158.202.56:443
http://51.159.226.129:443
http://51.158.253.130:14991
http://51.159.225.194:23767
http://51.158.204.131:23767
http://174.138.54.65:443
http://195.231.69.203:443
http://51.158.253.45:7901
http://47.242.121.18:8443
http://167.103.6.35:10230
http://51.158.253.115:17401
http://104.129.194.43:11639
http://23.94.251.185:3128
http://51.159.226.154:17401
http://165.225.72.38:11687
http://104.129.194.38:11658
http://185.53.129.141:9090
http://51.158.202.102:8118
http://51.159.225.202:8118
http://51.159.226.149:7081
http://104.129.194.46:11698
http://51.158.252.3:443
http://51.159.226.159:10243
http://104.129.194.44:11691
http://165.225.72.38:11679
http://104.129.194.46:11721
http://23.106.249.34:21125
http://51.158.252.160:10074
http://104.129.194.43:11643
http://104.129.194.44:11639
http://104.129.194.38:11639
http://51.158.253.152:10243
http://51.159.226.151:10074
http://23.106.56.19:7579
http://51.159.226.159:5094
http://104.129.194.46:11730
http://51.159.226.128:15846
http://51.158.252.50:15846
http://23.106.56.11:7714
http://104.129.194.38:11734
http://154.26.182.140:3128
http://23.106.56.35:16927
http://62.171.146.164:443
http://23.106.249.54:6550
http://51.159.225.196:23856
http://107.152.33.202:8888
http://66.135.16.53:443
http://23.106.56.52:15767
http://51.159.225.197:19362
http://103.37.111.253:10088
http://104.129.194.43:11654
http://104.129.194.45:11654
http://51.158.204.131:3004
http://151.115.98.4:443
http://51.159.226.153:7901
http://23.106.56.54:6157
http://51.158.252.149:16684
http://51.159.226.126:16684
http://109.123.219.11:8689
http://51.158.253.45:5299
http://104.129.194.46:11718
http://51.158.204.46:21384
http://51.159.226.161:19030
http://51.159.226.156:10472
http://209.13.186.20:443
http://51.158.253.152:5698
http://51.159.226.159:5698
http://51.159.226.126:443
http://193.176.84.18:9002
http://193.176.84.5:9002
http://193.176.84.25:9002
http://193.176.84.27:9002
http://193.176.84.31:9002
http://151.115.98.9:443
http://23.106.249.52:2569
http://151.115.98.6:443
http://104.129.194.38:11737
http://151.115.98.198:443
http://5.189.184.6:443
http://108.181.18.175:4001
http://104.129.194.38:11721
http://47.243.181.85:41713
http://151.115.98.120:443
http://47.243.181.85:8080
http://104.129.194.45:11730
http://51.159.28.39:443
http://104.129.194.43:10107
http://165.225.72.38:11677
http://104.129.194.46:11681
http://104.129.194.38:11643
http://104.129.194.45:11699
http://104.129.194.38:11748
http://104.129.194.43:11737
http://104.129.194.38:11699
http://104.129.194.45:11745
http://151.115.98.141:443
http://104.129.194.46:11745
http://104.129.194.43:11745
http://104.129.194.43:11659
http://163.172.53.142:443
http://103.37.111.253:10091
http://196.1.93.10:443
http://151.115.98.201:443
http://151.115.98.213:443
http://47.243.181.85:41716
http://151.115.98.36:443
http://95.66.244.250:443
http://47.243.181.85:41688
http://163.172.167.48:443
http://109.123.219.11:8686
http://180.166.128.182:12011
http://151.115.98.203:443
http://51.159.226.148:443
http://151.115.98.8:443
http://95.164.1.87:443
http://94.16.116.86:8448
http://165.225.72.38:11731
http://51.159.226.125:443
http://47.243.181.85:41698
http://205.178.137.173:8447
http://151.115.99.132:443
http://151.115.98.18:443
http://151.115.98.7:443
http://51.158.202.56:14287
http://51.158.194.107:14287
http://51.158.194.16:14287
http://45.12.132.12:8081
http://45.12.136.164:8081
http://5.182.37.132:8081
http://5.182.39.182:8081
http://89.111.26.157:443
http://51.159.226.163:443
http://165.225.72.38:11011
http://104.129.194.43:11011
http://104.129.194.38:11655
http://89.111.26.249:443
http://151.115.98.205:443
http://158.173.154.3:9002
http://158.173.154.46:9002
http://158.173.154.62:9002
http://158.173.154.64:9002
http://158.173.154.65:9002
http://158.173.154.74:9002
http://158.173.154.80:9002
http://82.115.60.66:443
http://89.111.26.123:443
http://180.166.128.182:24001
http://89.111.28.77:443
http://104.129.194.38:11692
http://89.111.26.242:443
http://180.166.128.182:34000
http://89.111.26.246:443
http://89.111.26.243:443
http://89.111.26.218:443
http://205.178.137.221:8447
http://165.225.72.38:11722
http://165.225.72.38:11706
http://104.129.194.46:11662
http://104.129.194.45:11697
http://104.129.194.44:11771
http://134.65.238.25:3128
http://104.129.194.46:11727
http://47.243.181.85:41804
http://104.129.194.45:11748
http://104.129.194.46:11682
http://205.178.144.167:8447
http://47.243.181.85:55011
http://104.129.194.44:11065
http://205.178.137.250:8443
http://104.129.194.38:11731
http://205.178.137.6:8443
http://47.243.181.85:41407
http://47.243.181.85:41398
http://89.111.26.250:443
http://203.115.101.58:443
http://89.111.26.217:443
http://192.210.193.125:3129
http://47.243.181.85:41131
http://47.243.181.85:41289
http://205.178.137.151:8443
http://47.243.181.85:41312
http://65.108.209.73:443
http://47.243.181.85:28085
http://158.173.154.55:9002
http://62.84.245.79:443
http://51.159.226.124:443
http://89.111.28.83:443
http://210.209.236.242:443
http://87.95.146.131:443
http://144.202.113.89:443
http://60.244.184.155:443
http://82.115.60.51:443
http://89.111.31.104:443
http://75.84.71.14:443
http://20.81.205.173:443
http://47.243.181.85:41799
http://165.227.118.27:443
http://182.155.200.61:443
http://205.178.136.191:8443
http://20.78.213.56:443
http://119.194.72.67:443
http://88.99.99.26:8888
http://65.108.103.19:443
http://212.47.232.28:443
http://43.224.4.109:443
http://211.250.42.219:443
http://89.111.31.242:443
http://47.243.181.85:41405
http://147.231.163.133:443
http://158.173.154.82:9002
http://158.173.154.83:9002
http://158.173.154.77:9002
http://158.173.154.79:9002
http://158.173.154.85:9002
http://165.225.122.32:10230
http://158.173.154.75:9002
http://104.129.194.38:11646
http://165.232.97.74:443
http://89.111.31.237:443
http://181.16.201.58:8091
http://104.129.194.46:11746
http://104.129.194.44:11746
http://125.59.132.156:443
http://185.88.141.109:443
http://143.198.135.176:443
http://139.162.236.244:443
http://51.159.225.194:3004
http://221.153.152.43:443
http://85.214.204.79:443
http://62.99.135.51:443
http://41.139.151.147:443
http://119.236.179.153:443
http://70.166.69.30:443
http://150.136.163.51:443
http://51.222.85.252:443
http://89.111.26.244:443
http://89.111.26.247:443
http://58.182.198.111:443
http://89.238.200.81:443
http://165.225.113.220:11717
http://165.225.113.220:11741
http://165.225.113.220:11657
http://158.173.154.81:9002
http://165.225.113.220:11749
http://165.225.113.220:11687
http://165.225.113.220:11725
http://165.225.113.220:11715
http://165.225.113.220:11677
http://165.225.113.220:10228
http://165.225.113.220:11742
http://47.243.181.85:41295
http://47.243.181.85:41310
http://165.225.113.220:11703
http://165.225.113.220:11273
http://165.225.113.220:11747
http://47.243.181.85:41316
http://165.225.113.220:11692
http://205.178.137.169:8443
http://47.243.181.85:41318
http://47.243.181.85:41287
http://165.225.113.220:11265
http://205.178.137.88:8447
http://47.243.181.85:41302
http://165.225.113.220:11716
http://205.178.136.3:8443
http://47.243.181.85:41303
http://47.243.181.85:41305
http://181.16.201.58:8097
http://165.225.113.220:11480
http://47.243.181.85:41311
http://47.243.181.85:41296
http://104.129.194.46:11643
http://47.243.181.85:41290
http://192.151.153.243:443
http://205.178.144.247:8447
http://206.188.209.77:8447
http://205.185.125.136:8888
http://104.129.194.46:11888
http://104.230.113.255:443
http://45.89.220.78:443
http://195.123.211.248:443
http://165.225.113.220:11681
http://114.55.114.43:8441
http://205.178.186.32:8443
http://205.178.186.177:8443
http://89.111.30.89:443
http://37.203.37.86:443
http://114.55.114.43:8001
http://114.55.114.43:8250
http://39.109.130.95:443
http://89.111.31.102:443
http://37.203.37.94:443
http://205.178.186.25:8443
http://205.178.186.190:8443
http://205.178.137.215:8447
http://206.188.212.169:8447
http://205.178.137.140:8443
http://205.178.144.210:8443
http://205.178.144.147:8447
http://89.111.31.245:443
http://104.129.194.45:11737
http://86.123.65.26:443
http://146.19.9.236:443
http://205.178.186.72:8443
http://176.226.200.167:443
http://89.111.30.182:443
http://205.178.186.185:8447
http://45.134.217.27:8443
http://37.203.37.92:443
http://47.243.181.85:41130
http://107.150.52.100:443
http://89.111.31.249:443
http://43.198.152.205:8890
http://139.162.16.201:443"""

In [18]:
import httpx


In [26]:
#load this into a list
url_list="https://cdn.jsdelivr.net/gh/proxifly/free-proxy-list@main/proxies/protocols/http/data.txt"

data=httpx.get(url_list).text.splitlines()

len(data)


2763

In [38]:

pp=['http://47.56.110.204:8989',
 'http://203.19.38.114:1080',
 'http://123.30.154.171:7777',
 'http://47.91.65.23:3128',
 'http://84.39.112.144:3128',
 'http://93.171.157.249:8080',
 'http://114.130.153.122:58080',
 'http://142.93.202.130:3128',
 'http://103.106.112.166:1234',
 'http://81.169.213.169:8888',
 'http://31.10.83.158:8080',
 'http://176.31.69.137:56789',
 'http://47.89.184.18:3128',
 'http://197.164.101.11:1981',
 'http://160.251.142.232:80',
 'http://103.155.62.141:8081',
 'http://87.239.31.42:80',
 'http://103.138.145.34:80',
 'http://103.178.86.10:8080',
 'http://124.108.6.20:8085',
 'http://2.57.218.130:8080',
 'http://103.231.236.239:8182',
 'http://103.231.236.227:8182',
 'http://190.12.150.244:999',
 'http://103.153.38.105:8083',
 'http://103.194.175.51:7777',
 'http://45.177.16.137:999',
 'http://190.242.157.215:8080',
 'http://195.62.50.74:8080',
 'http://58.216.109.18:800',
 'http://134.249.185.223:41890',
 'http://103.36.10.254:8088',
 'http://213.142.156.97:80',
 'http://187.102.219.42:999',
 'http://187.102.219.64:999',
 'http://176.117.106.149:8080',
 'http://182.253.36.151:8080',
 'http://27.49.68.66:9999',
 'http://187.49.191.13:999',
 'http://41.128.72.147:1981',
 'http://103.97.140.99:1080',
 'http://97.78.58.180:8080',
 'http://175.106.15.189:8080',
 'http://103.144.102.245:8080',
 'http://103.167.171.151:8097',
 'http://190.60.61.42:8080',
 'http://103.177.20.21:8181',
 'http://103.172.121.26:8080',
 'http://167.249.52.225:999',
 'http://183.110.216.128:8090',
 'http://200.48.35.122:999',
 'http://186.31.135.201:999',
 'http://210.87.74.85:9090',
 'http://183.110.216.159:8091',
 'http://217.113.25.196:8080',
 'http://103.171.183.255:1111',
 'http://103.15.214.20:8080',
 'http://200.59.191.233:999',
 'http://200.59.191.234:999',
 'http://210.209.236.242:80',
 'http://34.146.60.198:80',
 'http://85.220.141.220:80',
 'http://103.130.197.36:80',
 'http://23.190.168.41:80',
 'http://102.223.9.53:80',
 'http://185.135.69.34:80',
 'http://212.114.194.73:80',
 'http://81.10.234.234:80',
 'http://8.212.177.126:8080',
 'http://176.126.164.213:80',
 'http://101.50.101.91:80',
 'http://212.114.194.72:80',
 'http://23.148.152.46:80',
 'http://172.238.22.205:80',
 'http://185.187.92.42:80',
 'http://62.113.119.14:8080',
 'http://103.125.31.222:80',
 'http://27.34.242.98:80',
 'http://82.115.25.50:80',
 'http://58.152.58.208:80',
 'http://51.141.175.118:80',
 'http://154.211.2.68:80',
 'http://34.120.39.81:80',
 'http://40.89.145.14:80',
 'http://161.248.25.2:8080',
 'http://45.59.186.60:80',
 'http://178.212.144.7:80',
 'http://93.115.240.28:80',
 'http://154.31.115.210:80',
 'http://35.202.49.74:80',
 'http://4.213.167.178:80',
 'http://196.1.97.198:80',
 'http://34.44.49.215:80',
 'http://172.200.72.48:80',
 'http://46.62.171.144:80',
 'http://123.205.71.210:80',
 'http://174.136.204.40:80',
 'http://52.229.30.3:80',
 'http://95.216.49.153:80',
 'http://43.200.30.144:80',
 'http://4.213.98.253:80',
 'http://74.176.195.135:80',
 'http://91.107.171.19:80',
 'http://48.210.225.96:80',
 'http://154.17.224.118:80',
 'http://41.220.16.214:80',
 'http://89.116.88.19:80',
 'http://197.221.249.195:80',
 'http://46.62.254.141:80',
 'http://157.180.118.86:80',
 'http://154.31.113.209:80',
 'http://5.161.155.252:80',
 'http://103.253.43.144:80',
 'http://65.109.176.217:80',
 'http://202.83.30.96:80',
 'http://172.193.178.226:80',
 'http://59.6.25.118:3128',
 'http://62.182.204.81:88',
 'http://119.148.39.29:8444',
 'http://188.191.165.159:70',
 'http://113.192.12.73:8085',
 'http://38.60.196.214:80',
 'http://128.199.1.8:3128',
 'http://13.80.248.145:3128',
 'http://103.118.44.217:8080',
 'http://212.114.194.76:80',
 'http://162.245.85.36:80',
 'http://162.223.90.144:80',
 'http://103.180.126.166:8080',
 'http://154.56.114.10:8082',
 'http://101.47.16.15:7890',
 'http://104.197.218.238:8080',
 'http://101.255.53.105:8080',
 'http://154.3.236.202:3128',
 'http://185.36.193.29:80',
 'http://194.26.141.202:3128',
 'http://181.78.202.28:8080',
 'http://177.242.137.182:999',
 'http://120.71.1.56:8093',
 'http://58.69.209.71:8082',
 'http://39.109.113.97:4090',
 'http://45.166.72.35:999',
 'http://205.164.192.115:999',
 'http://163.5.128.221:14270',
 'http://181.78.9.50:999',
 'http://173.249.53.172:8080',
 'http://3.37.191.16:80',
 'http://54.201.87.119:80',
 'http://174.138.167.250:8080',
 'http://18.202.158.161:80',
 'http://156.146.56.232:8081',
 'http://94.182.177.92:80',
 'http://128.199.202.122:8080',
 'http://46.161.6.165:8080',
 'http://51.8.61.60:80',
 'http://46.62.225.43:80',
 'http://154.16.146.45:80',
 'http://104.168.10.224:8888',
 'http://113.200.109.19:80',
 'http://133.18.234.13:80',
 'http://147.231.163.133:80',
 'http://139.162.78.109:8080',
 'http://113.108.13.120:8083',
 'http://183.215.23.242:9091',
 'http://113.204.79.230:9091',
 'http://35.197.89.213:80',
 'http://61.158.175.38:9002',
 'http://192.73.244.36:80',
 'http://111.120.8.205:8085',
 'http://58.241.88.18:800',
 'http://12.50.107.222:80',
 'http://154.205.147.214:80',
 'http://154.17.228.122:80',
 'http://58.216.109.14:800',
 'http://79.175.188.151:80',
 'http://139.59.1.14:8080',
 'http://221.231.13.198:1080',
 'http://159.65.245.255:80',
 'http://156.146.56.240:8082',
 'http://120.92.212.16:7890',
 'http://23.247.136.254:80',
 'http://139.99.237.62:80',
 'http://89.58.57.45:80',
 'http://188.40.57.101:80',
 'http://5.129.246.218:1081',
 'http://12.50.107.219:80',
 'http://41.191.203.162:80',
 'http://41.220.16.213:80',
 'http://59.7.246.4:80',
 'http://89.58.55.33:80',
 'http://103.94.52.70:3128',
 'http://134.209.29.120:80',
 'http://138.68.60.8:80',
 'http://139.162.200.213:80',
 'http://143.42.66.91:80',
 'http://154.61.76.24:8081',
 'http://161.35.70.249:80',
 'http://185.36.145.215:80',
 'http://190.116.28.148:80',
 'http://195.114.209.50:80',
 'http://197.221.240.247:80',
 'http://212.114.194.78:80',
 'http://16.163.88.228:80',
 'http://115.239.234.43:7302',
 'http://159.203.61.169:3128',
 'http://198.199.86.11:3128']

In [47]:
import threading

def add_one(x):
    return x+1


x=threading.Thread(target=add_one, args=(5,))
x.start()

In [ ]:
from flask_app import  app as appx

SyntaxError: invalid syntax (3789886372.py, line 1)

In [43]:
import joblib
from joblib_progress import joblib_progress

def test_proxy(i):
    with httpx.Client(proxy=i) as client: 
        try:
            response = client.get("https://rubinothings.com.br/status",timeout=10)
            if response.status_code==200:
                print(f"proxy found: {i}")
                return {"status":"200","proxy":i}
            else:
                print(f"proxy failed: {i}")
                return {"status":"non-200","proxy":i}
        except Exception as e:
            print(f"proxy error: {i} - {e}")
            return {"status":"error","proxy":i}
#run in parallel  32 threads

with joblib_progress("fetching proxies", total=len(pp)):

    r=joblib.Parallel(n_jobs=32)(joblib.delayed(test_proxy)(i) for i in pp)


Output()

In [44]:
r

[{'status': 'error', 'proxy': 'http://47.56.110.204:8989'},
 {'status': 'error', 'proxy': 'http://203.19.38.114:1080'},
 {'status': 'error', 'proxy': 'http://123.30.154.171:7777'},
 {'status': 'error', 'proxy': 'http://47.91.65.23:3128'},
 {'status': 'error', 'proxy': 'http://84.39.112.144:3128'},
 {'status': 'error', 'proxy': 'http://93.171.157.249:8080'},
 {'status': 'error', 'proxy': 'http://114.130.153.122:58080'},
 {'status': 'error', 'proxy': 'http://142.93.202.130:3128'},
 {'status': 'error', 'proxy': 'http://103.106.112.166:1234'},
 {'status': 'error', 'proxy': 'http://81.169.213.169:8888'},
 {'status': 'error', 'proxy': 'http://31.10.83.158:8080'},
 {'status': 'error', 'proxy': 'http://176.31.69.137:56789'},
 {'status': 'error', 'proxy': 'http://47.89.184.18:3128'},
 {'status': 'error', 'proxy': 'http://197.164.101.11:1981'},
 {'status': 'error', 'proxy': 'http://160.251.142.232:80'},
 {'status': '200', 'proxy': 'http://103.155.62.141:8081'},
 {'status': 'error', 'proxy': 'htt

In [45]:
pt=[t['proxy'] for t in r if t["status"]=="200"]

In [46]:
pt

['http://103.155.62.141:8081',
 'http://45.177.16.137:999',
 'http://190.242.157.215:8080',
 'http://187.102.219.64:999',
 'http://41.128.72.147:1981',
 'http://62.113.119.14:8080',
 'http://59.6.25.118:3128',
 'http://101.47.16.15:7890',
 'http://154.3.236.202:3128',
 'http://194.26.141.202:3128',
 'http://205.164.192.115:999']

In [36]:
url = "https://rubinothings.com.br/status"

url = "http://httpbin.org/ip"

raw=p
used=[]



import random 
rawt=[] #debug
for i in rawt:
    with httpx.Client(proxy=i) as client:
        try:
            response = client.get("https://httpbin.org/ip",timeout=10)
            if response.status_code==200:
                used.append(i)
                print(f"proxy found: {i}")
                
            else:
                print(f"proxy failed: {i}")
        except Exception as e:
            print(f"proxy error: {i} - {e}")


import joblib
from joblib_progress import joblib_progress

def test_proxy(i):
    with httpx.Client(proxy=i) as client: 
        try:
            response = client.get("http://httpbin.org/ip",timeout=10)
            if response.status_code==200:
                print(f"proxy found: {i}")
                print(response.text)
                return {"status":"200","proxy":i}
            else:
                print(f"proxy failed: {i}")
                return {"status":"non-200","proxy":i}
        except Exception as e:
            print(f"proxy error: {i} - {e}")
            return {"status":"error","proxy":i}
#run in parallel  32 threads

test_proxy(raw[1])


proxy found: http://203.19.38.114:1080
{
  "origin": "203.19.38.114"
}



{'status': '200', 'proxy': 'http://203.19.38.114:1080'}

In [17]:
r

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [ ]:
# ANALYTICS AND REPORTING EXAMPLES
from analytics import get_character_summary, print_character_report, calculate_xp_growth

# Get comprehensive character summary
summary = get_character_summary(db_manager, "rollabostx")

# Print formatted report
print_character_report(db_manager, "rollabostx")

# Calculate specific metrics
xp_growth_7d = calculate_xp_growth(db_manager, 1, days_back=7)  # character_id=1, last 7 days
print(f"\n📈 7-Day XP Growth: {xp_growth_7d}")

In [ ]:
# BULK CHARACTER PROCESSING
# Example of how to process multiple characters at once

def process_multiple_characters(character_list, world="Auroria"):
    """
    Process multiple characters and store their data
    """
    results = {}
    
    for character_name in character_list:
        print(f"\n🔄 Processing {character_name}...")
        result = scrape_and_store_character(character_name, world)
        results[character_name] = result
        
        # Small delay to be respectful to the server
        import time
        time.sleep(2)  # 2 second delay between requests
    
    return results

# Example character list
character_list = ["rollabostx", "another_character", "third_character"]  # Add your character names

# Uncomment to run bulk processing
# bulk_results = process_multiple_characters(character_list)
# print(f"\n📊 Bulk Processing Results:")
# for char, result in bulk_results.items():
#     if result:
#         total_new = sum([result['new_deaths'], result['new_kills'], 
#                         result['new_online_times'], result['new_experiences']])
#         print(f"   {char}: {total_new} new records")
#     else:
#         print(f"   {char}: Failed to process")

print("💡 Uncomment the bulk processing section above to process multiple characters")